## IODA Architektur

Die Radikale Objektorientierung mit ihren Prinzipien PoMO und IOSP definiert für Flow-Design die grundlegende Struktur von Software. *Messaging* ist das Paradigma für die Kommunikation zwischen Funktionseinheiten. Software steht damit dem Lebendigen näher als dem Mechanischen: Software wächst aus einer Zelle heran zu einem komplexen Gebilde, das sich ständig neuen Anforderungen anpasst.

### Wachsende Differenzierung

Flow-Design versteht Software quasi als "Schau", dessen "Blasen" sich immer weiter differenzieren. Alles beginnt mit einer "Blase" "für alles". Sie steht für eine komplette Problemlösung, ein gewünschtes Gesamtverhalten. Sie leistet mit ihrer Logik alle geforderten Nachrichtentransformationen.

/// 1 blase mit input u output (1)

Und diese Blase differenziert sich dann, wenn die Anforderungen an sie wachsen. Zuerst zerfällt sie vielleicht in zwei oder drei kleinere Blasen:

/// 1 blase differenziert sich in 2-3 kleinere. farblich unterschiedlich. (2)

Dann differenzieren sich die weiter in jeweils zwei, drei, vier noch spezifischere Blasen:

/// die kleineren blasen zerfallen weiter in je 2-4 kleinere (3)

Und so weiter und so fort...

/// ein objektschaut mit vielen größeren und kleineren blasen, die alle wieder weitere enthalten. (4)

Die Verbindungen der Blasen in Form von Nachrichtenkanälen sind in diesem Bild der Einfachheit halber ausgeblendet. Dass zwischen den Blasen in den Strata nur unidirektional Daten fließen und die Blasen einander nicht kennen und dass nur die innersten Blasen Logik enthalten, ist schon ausführlich beschrieben worden. Das Bild vom Schaum soll vielmehr den Gedanken des Wachstums vermitteln: Logik in Software nimmt ständig an Menge zu. Neue Features verändern Logik und fügen vor allem neue hinzu. Um bei dem Wachstum den Überblick zu behalten, um produktiv zu bleiben, ist Logik zu getrennten Verantwortlichkeiten zusammenzufassen. Das ist mit Differenzierung gemeint.

#### Funktionen als Container für Funktionalität

Zuerst findet eine funktionale Differenzierung statt. Logik, die bisher "auf einem Haufen gelegen hat", wird auf verschiedene "Haufen" verteilt. Das geschieht im Code mittels Funktionen als Container; zunächst entstehen also Operationen. Jede Operation stellt einen anderen Teil des Verhaltens her. Integrationen komponieren aus Operationen mit ganz unterschiedlicher Logik für ganz unterschiedliche Verantwortlichkeiten dann höhere Funktionseinheiten, deren Details im Weiteren uninteressant sind. Eine Integration stellt für andere Funktionseinheiten eine Black Box dar.

Das schrittweise Wachsen und Ausdifferenzieren von Logik in Funktionen hier an einem kleinen Beispiel. Alles beginnt ganz einfach mit einer Anforderung, zu deren Erfüllung nur wenig Logik nötig ist. Doch dann kommen weitere Anforderungen hinzu und die Software soll "in geordneter Weise" wachsen:

In [11]:
// Anforderung: einziffrige römische Zahlen konvertieren

int FromRoman_v1(string roman) { // Operation
    return roman[0] switch {
        'I' => 1, 'V' => 5, 'X' => 10,
        'L' => 50, 'C' => 100, 'D' => 500,
        'M' => 1000
    };
}

display(FromRoman_v1("X"));

10

In [15]:
// Anforderung: mehrziffrige römische Zahlen konvertieren (ohne Subtraktionsregel)
// Die Logik wächst!

int FromRoman_v2(string roman) { // Operation
    var dezimal = 0;
    foreach(var romanDigit in roman.ToCharArray())
        dezimal += romanDigit switch {
            'I' => 1, 'V' => 5, 'X' => 10,
            'L' => 50, 'C' => 100, 'D' => 500,
            'M' => 1000
        };
    return dezimal;
}

display(FromRoman_v2("XVI"));

16

In [37]:
// Ausdifferenzierung der Logik in Funktionen
// Refaktorisierung!

int FromRoman_v3(string roman) { // Integration
    var values = Map(roman);
    return Sum(values);
}

int[] Map(string roman) // Operation
    => roman.ToCharArray()
            .Select(romanDigit => romanDigit switch {
                        'I' => 1, 'V' => 5, 'X' => 10,
                        'L' => 50, 'C' => 100, 'D' => 500,
                        'M' => 1000
                    })
            .ToArray();

int Sum(IEnumerable<int> values) { // Operation
    var sum = 0;
    foreach(var v in values) sum += v;
    return sum;
}


display(FromRoman_v3("CLXVI"));

166

In [35]:
// Anforderung: Subtraktionsregel beachten
// Die Logik wächst!

int FromRoman_v4(string roman) { // Widerspruch zum IOSP! Hybride Funktion: operiert und integriert
    var values = Map(roman);
    for(var i=0; i<values.Length-1; i++)
        if (values[i]<values[i+1])
            values[i] *= -1;
    return Sum(values);
}

display(FromRoman_v4("XLII"));

42

In [36]:
// Gewachsene Logik weiter in Funktionen ausdifferenzieren
// Refaktorisierung!

int FromRoman_v5(string roman) { // Integration
    var values = Map(roman);
    values = Negate(values);
    return Sum(values);
}

int[] Negate(int[] values) { // Operation
    var negatedValues = (int[])values.Clone();
    for(var i=0; i<negatedValues.Length-1; i++)
    if (negatedValues[i]<negatedValues[i+1])
        negatedValues[i] *= -1;
    return negatedValues;
}

display(FromRoman_v5("MCMLXXXIV"));

1984

Selbstverständlich hätte die ganze Logik in einer Funktion `FromRoman` anwachsen können. Das wäre das übliche Vorgehen - "bis es irgendwie nicht mehr geht". Aber wann ist der Punkt erreicht, "dass es nicht mehr geht"? Flow-Design ist da sehr rigoros mit dem IOSP: Sobald auch nur ein Teil von Logik aus einer Operation in eine Funktion extrahiert wird, muss die bisherige Operation "umkippen" und zu einer Integration werden. Die Refaktorisierung nach `FromRoman_v3` hat sofort das IOSP beachtet. Um schnell das neue Feature herzustellen, hat `FromRoman_v4` jedoch die weitere Logik wider das IOSP zwischen die Funktionsaufrufe geschoben. Das hat `FromRoman_v5` mit einer Refaktorisierung ausgleichen müssen.

Das IOSP sorgt mithin dafür, dass unterschiedliche Verantwortlichkeiten sich in einem streng geordneten in der Breite und in der Tiefe wachsenden Funktionsbaum ausdifferenzieren. An der Wurzel steht eine Funktion, die umfassende Verhaltensanforderungen repräsentiert. Darunter hängen Funktionen, die feinere und feinere Teilaspekte erfüllen. Nur die Blätter dieses Baumes enthalten schwer verständliche Logik - allerdings in nur kleinen Mengen.

Ohne IOSP können Funktionen beliebig im Umfang wachen. Vorhandener Logik kann weitere hinzugefügt werden; falls Logik unübersichtlich wird, kann ein Teil ausgelagert werden in eine funktionale Abhängigkeit usw. usf. Mit dem IOSP jedoch wachsen Operationen nicht beliebig. Sobald nach vielleicht 20 oder 50 Zeilen Logik die Unverständlichkeit genügend gewachsen ist, muss eine Operation umkippen in eine Integration, sobald auch nur ein Teil der Logik extrahiert wird. Integrationen wachsen auch nicht beliebig, weil schon nach 10 oder 20 Zeilen genügend Funktionsaufrufe beieinander sind, um Teile davon zu extrahieren in eine untergeordnete Integration.

Das IOSP sorgt sozusagen dafür, dass die Blasen im Objektschaum immer wieder platzen.

#### Module als Container für Zwecke

Alan Kay hat den Begriff Objekt geprägt. In seiner Analogie war von Klassen nicht die Rede. Objekte sind die Funktionseinheiten, die etwas leisten. Die Übersetzung dafür im Code sind Funktionen, die Logik enthalten. Jede steht für einen Teil einer Laufzeitqualität. Was hat es dann aber mit Klassen auf sich?

Wachsende Logik differenziert das IOSP in eine wachsende Zahl von Funktionen. Integrationen ordnen darin Operationen. Integrationen abstrahieren also die Verhaltensproduktion. Integrationen komponieren aus Verschiedenem ein neues Ganzes.

Doch die wachsende Zahl von Funktionen - so verständlich und testbar sie Logik kapseln mögen oder als Integrationen nicht einmal enthalten - wird mit der Zeit auch unhandlich. Es braucht ein Ordnungsmittel, um darin den Überblick zu behalten.

Wie sich bei näherer Betrachtung zeigt, sind zwar alle Funktionen verschieden in ihrem Verhalten, doch sie gleichen sich in ihrem Zweck. Neben der funktionalen Differenzierungsdimension des Schaums gibt es eine weitere, die des Zwecks. Orthogonal zum Funktionsbaum wächst ein zweiter Baum, der der Module.

> **Module fassen Funktionen nach Ähnlichkeit, nach Gemeinsamkeit zusammen.**

Module sind Container wie Funktionen und Hosts:

* Funktionen mit ihrer direkten und indirekten Integration von Logik dienen der Herstellung funktionaler und nicht-funktionaler Qualitäten.
* Hosts beherbergen Funktionen zur Laufzeit, um weitere nicht-funktionale Qualitäten herzustellen.
* Module enthalten Funktionen zur Entwicklungszeit, um die Produktivitätsqualität zu steigern.

/// im schaum blasen farblich zu modulen zusammenfassen. evtl jetzt den kreis mit einer farbe und den inhalt mit einer anderen? (5)

Nach welchen Kriterien Module Funktionen zusammenfassen, sei zunächst dahingestellt. Wesentlich ist, dass ihre Abstraktion eine Aggregation darstellt. Das Motto ist "Gleich und Gleich gesellt sich gern." Integrationen hingegen abstrahierenden Unterschiedliches.

Klassen sind nur die unterste Ebene einer Modulhierarchie, die in Erklärungen zum Entwurf noch näher definiert wird. An dieser Stelle soll genügen, Module sehr allgemein als Sammlungen von Funktionen einzuführen, deren Hierarchie orthogonal zur Funktionshierarchie ist.

Als Beispiel mag das Modell für diese Anforderungen dienen:

> CSV-Daten sollen in eine simple ASCII-Tabelle gewandelt werden.
> Die zugehörige Funktion soll so aussehen: `string Tabellieren(string csv)`

Beispiel für CSV-Daten:

```
Name;Strasse;Ort;Alter
Peter Pan;Am Hang 5;12345 Einsam;42
Maria Schmitz;Kölner Straße 45;50123 Köln;43
Paul Meier;Münchener Weg 1;87654 München;65
```

Beispiel für die daraus zu erzeugende ASCII-Tabelle:

```
Name         |Strasse         |Ort          |Alter|
-------------+----------------+-------------+-----+
Peter Pan    |Am Hang 5       |12345 Einsam |42   |
Maria Schmitz|Kölner Straße 45|50123 Köln   |43   |
Paul Meier   |Münchener Weg 1 |87654 München|65   |
```

Das Laufzeitmodell zur Lösung der Anforderungen könnte rein aus Operationen bestehen:

/// csv tabellieren in einem fluss (6)

Mehr Übersichtlichkeit bietet jedoch eine Hiearchie von Funktionseinheiten:

/// csv tabellieren in einer hierarchie (7)

Jede Funktionseinheit kann als Funktion umgesetzt werden. Mit solcher Differenzierung wäre die Logik verständlich und gut testbar gegliedert. Aber die große Zahl an Funktionen wäre womöglich unübersichtlich. Eine Modularisierung würde helfen, Ordnung in die Funktionen zu bringen.

/// csv tabellieren in der hierarchie mit klassen (8)

Der Funktionshierarchie stünde damit eine Modulhierarchie gegenüber:

/// csv datenfluss als abhängigkeitshierarchie vs klassenhierarchie (9)

## System vs Umwelt
bisher system - aber was ist drumherum?
user und ressourcen

hier kommt zustand auch ins spiel

## Softwarezelle
### Adapter
### Domäne
### Integration